In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [76]:
podatki_iz_turnirjev = pd.read_csv("podatki_iz_turnirjev.csv")
podatki_o_narodnosti = pd.read_csv("podatki_o_narodnosti.csv")
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(podatki_iz_turnirjev["tip_zmage"])
podatki_iz_turnirjev

,leto,mesec,dan,turnir,igralec_crni,igralec_crni_rang,igralec_beli,igralec_beli_rang,zmagovalec,tip_zmage
0,1969,12,29,"1st All-Japan First Place, challenger decision...",Fujisawa Hideyuki,9.0,Sugiuchi Masao,9.0,B,8.5
1,1969,12,26,"17th Japanese NHK Cup, round 2",Go Seigen,9.0,Fujisawa Hideyuki,9.0,W,8.5
2,1969,12,24,"9th Japanese Old Meijin, league",Honda Kunihisa,8.0,Takagawa Kaku,9.0,B,3
3,1969,12,24,"8th Japanese Judan, title match #3",Sakata Eio,9.0,Otake Hideo,8.0,W,R
4,1969,12,18,"9th Japanese Judan, preliminary",Kudo Norio,8.0,Takemiya Masaki,5.0,W,R
...,...,...,...,...,...,...,...,...,...,...
104503,2022,1,1,"Year 2021 Korean League, round 6",Choi Cheolhan,9.0,Park Yeonghun,9.0,W,R
104504,2022,1,1,"Year 2021 Korean League, round 6",Na Hyun,9.0,Byun Sangil,9.0,B,R
104505,2022,1,1,"Year 2021 Korean League, round 6",Park Geunho,5.0,Park Junghwan,9.0,W,R
104506,2022,1,1,"Year 2021 Korean League, round 6",Kim Jinhwi,5.0,Lee Changseok,8.0,W,R


In [66]:
podatki_iz_turnirjev['igralec_crni_rang'] = podatki_iz_turnirjev['igralec_crni_rang'].fillna(0)
podatki_iz_turnirjev['igralec_beli_rang'] = podatki_iz_turnirjev['igralec_beli_rang'].fillna(0)
podatki_iz_turnirjev['tip_zmage'] = podatki_iz_turnirjev['tip_zmage'].fillna(0)
podatki_iz_turnirjev.leto = podatki_iz_turnirjev.leto.astype(int)
podatki_iz_turnirjev.mesec = podatki_iz_turnirjev.mesec.astype(int)
podatki_iz_turnirjev.dan = podatki_iz_turnirjev.dan.astype(int)
podatki_iz_turnirjev.igralec_beli_rang = podatki_iz_turnirjev.igralec_beli_rang.astype(int)
podatki_iz_turnirjev.igralec_crni_rang = podatki_iz_turnirjev.igralec_crni_rang.astype(int)

In [67]:
def dobi_leto(leto, podatki):
    relevantni_podatki = podatki[podatki.leto == leto]
    return relevantni_podatki

def izberi_polja(polja, podatki):
    return podatki[polja]

def dobi_stevilo_iger(podatki):
    return podatki.shape[0]

def izlusci_igralce(podatki):
    igralci = set()
    raw_igralci = izberi_polja(["igralec_crni", "igralec_beli"], podatki)
    for ind in raw_igralci.index:
        igralci.add(raw_igralci['igralec_crni'][ind].strip())
        igralci.add(raw_igralci['igralec_beli'][ind].strip())
    return list(igralci)

Preštej zmage igralcev (procent zmag čez čas)
Preštej zmage glede na državo in razliko v rangu
Preštej zmage glede na državo in barvo
Preštej zmage glede na barvo in razliko v rangu
Država glede na ime

In [91]:
bela_po_letih = []
crna_po_letih = []
for leto in range(1969, 2023):
    podatki = dobi_leto(leto, podatki_iz_turnirjev)
    podatki = podatki[podatki.zmagovalec != "Void"]
    podatki = izberi_polja(["zmagovalec"], podatki)
    stevilo_iger = dobi_stevilo_iger(podatki)
    podatki = podatki[podatki.zmagovalec == "W"]
    procent = dobi_stevilo_iger(podatki) / stevilo_iger
    bela_po_letih.append(procent)
    crna_po_letih.append(1 - procent)

In [ ]:
verjetnost = []
for leto in range(1969, 2023):
    podatki = dobi_leto(leto, podatki_iz_turnirjev)
    stevilo_iger = podatki[(podatki.igralec_crni_rang > podatki.igralec_beli_rang) | (podatki.igralec_crni_rang < podatki.igralec_beli_rang)].size
    podatki = izberi_polja(["igralec_crni_rang", "igralec_beli_rang", "zmagovalec"], podatki)
    verjetnost.append(podatki[
        (podatki.igralec_crni_rang > podatki.igralec_beli_rang) & (podatki.zmagovalec == "W")
        |
        (podatki.igralec_crni_rang < podatki.igralec_beli_rang) & (podatki.zmagovalec == "B")].size / stevilo_iger)

In [104]:
drzave_po_letih = []
for leto in range(1969, 2023):
    podatki = dobi_leto(leto, podatki_iz_turnirjev)
    podatki = izberi_polja(["igralec_crni", "igralec_beli", "zmagovalec"], podatki)
    for ind in podatki.index:
        if podatki["zmagovalec"][ind] == "W":
            igralec_z = podatki["igralec_beli"][ind]
            igralec_p = podatki["igralec_crni"][ind]
            drzava_z = podatki_o_narodnosti[podatki_o_narodnosti["Igralec"] == igralec_z]
            drzava_p = podatki_o_narodnosti[podatki_o_narodnosti["Igralec"] == igralec_p]
            if drzava_p != drzava_z:
                pass
